# Import needed modules

In [ ]:
#!pip install tensorflow==2.9.1

In [3]:
!pip install tensorflow

In [4]:
# import system libs
import os
import time
import shutil
import pathlib
import itertools

# import data handling tools
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# import Deep learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization, Layer 
from tensorflow.keras import regularizers
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

print ('modules loaded')

modules loaded


# Create needed functions

## Functions to Create Data Frame from Dataset

#### **Function to create data frame**

#### Function to generate images from dataframe

In [5]:
# Generate data paths with labels
def define_paths(data_dir):
    filepaths = []
    labels = []

    folds = os.listdir(data_dir)
    for fold in folds:
        foldpath = os.path.join(data_dir, fold)
        filelist = os.listdir(foldpath)
        for file in filelist:
            fpath = os.path.join(foldpath, file)
            filepaths.append(fpath)
            labels.append(fold)

    return filepaths, labels


# Concatenate data paths with labels into one dataframe ( to later be fitted into the model )
def define_df(files, classes):
    Fseries = pd.Series(files, name= 'filepaths')
    Lseries = pd.Series(classes, name='labels')
    return pd.concat([Fseries, Lseries], axis= 1)

# Split dataframe to train, valid, and test
def split_data(data_dir):
    # train dataframe
    files, classes = define_paths(data_dir)
    df = define_df(files, classes)
    strat = df['labels']
    train_df, dummy_df = train_test_split(df,  train_size= 0.8, shuffle= True, random_state= 123, stratify= strat)

    # valid and test dataframe
    strat = dummy_df['labels']
    valid_df, test_df = train_test_split(dummy_df,  train_size= 0.5, shuffle= True, random_state= 123, stratify= strat)

    return train_df, valid_df, test_df

In [6]:
def create_gens (train_df, valid_df, test_df, batch_size):
    '''
    This function takes train, validation, and test dataframe and fit them into image data generator, because model takes data from image data generator.
    Image data generator converts images into tensors. '''


    # define model parameters
    img_size = (224, 224)
    channels = 3 # either BGR or Grayscale
    color = 'rgb'
    img_shape = (img_size[0], img_size[1], channels)

    # Recommended : use custom function for test data batch size, else we can use normal batch size.
    ts_length = len(test_df)
    test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
    test_steps = ts_length // test_batch_size

    # This function which will be used in image data generator for data augmentation, it just take the image and return it again.
    def scalar(img):
        return img

    tr_gen = ImageDataGenerator(preprocessing_function= scalar, horizontal_flip= True)
    ts_gen = ImageDataGenerator(preprocessing_function= scalar)

    train_gen = tr_gen.flow_from_dataframe( train_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                        color_mode= color, shuffle= True, batch_size= batch_size)

    valid_gen = ts_gen.flow_from_dataframe( valid_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                        color_mode= color, shuffle= True, batch_size= batch_size)

    # Note: we will use custom test_batch_size, and make shuffle= false
    test_gen = ts_gen.flow_from_dataframe( test_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                        color_mode= color, shuffle= False, batch_size= test_batch_size)

    return train_gen, valid_gen, test_gen

#### **Function to display data sample**

In [7]:
def show_images(gen):
    '''
    This function take the data generator and show sample of the images
    '''

    # return classes , images to be displayed
    g_dict = gen.class_indices        # defines dictionary {'class': index}
    classes = list(g_dict.keys())     # defines list of dictionary's kays (classes), classes names : string
    images, labels = next(gen)        # get a batch size samples from the generator

    # calculate number of displayed samples
    length = len(labels)        # length of batch size
    sample = min(length, 25)    # check if sample less than 25 images

    plt.figure(figsize= (20, 20))

    for i in range(sample):
        plt.subplot(5, 5, i + 1)
        image = images[i] / 255       # scales data to range (0 - 255)
        plt.imshow(image)
        index = np.argmax(labels[i])  # get image index
        class_name = classes[index]   # get class of image
        plt.title(class_name, color= 'blue', fontsize= 12)
        plt.axis('off')
    plt.show()

#### **Callbacks** 
<br> 
Callbacks : Helpful functions to help optimize model training  <br> 
Examples: stop model training after specfic time, stop training if no improve in accuracy and so on.

In [8]:
import tensorflow as tf
import numpy as np
import time

class MyCallback(tf.keras.callbacks.Callback):
    # MyCallback sınıfını başlatır, eğitim sırasında kullanılacak parametreleri tanımlar
    def __init__(self, model, patience, stop_patience, threshold, factor, batches, epochs, ask_epoch):
        super(MyCallback, self).__init__()
        # Sabır değeri, öğrenme oranını düşürmeden önce kaç epoch beklenir
        self.patience = patience
        # Erken durdurma için sabır değeri, öğrenme oranı iyileşmezse eğitimi durdurmak için epoch sayısı
        self.stop_patience = stop_patience
        # Eğitim doğruluğu eşiği, bu eşikten düşükse accuracy izlenir, yüksekse val_loss izlenir
        self.threshold = threshold
        # Öğrenme oranını azaltma faktörü (örneğin, 0.5 ile yarıya indirir)
        self.factor = float(factor)
        # Her epoch'ta işlenecek batch sayısı
        self.batches = batches
        # Toplam eğitim epoch sayısı
        self.epochs = epochs
        # Kullanıcıya eğitimi durdurup durdurmayacağını sormak için epoch aralığı
        self.ask_epoch = ask_epoch
        # Başlangıç ask_epoch değerini saklar
        self.ask_epoch_initial = ask_epoch
        # En iyi ağırlıkları saklamak için
        self.best_weights = None
        # İlk ağırlıkları saklamak için
        self.initial_weights = None
        # En iyi performansı gösteren epoch numarası
        self.best_epoch = 1
        # Başlangıç öğrenme oranını modelden alır
        self.initial_lr = float(tf.keras.backend.get_value(model.optimizer.learning_rate))
        # En yüksek eğitim doğruluğu
        self.highest_tracc = 0.0
        # En düşük doğrulama kaybı
        self.lowest_vloss = float('inf')
        # Öğrenme oranı azaltma sayacı
        self.count = 0
        # Erken durdurma sayacı
        self.stop_count = 0

    # Sınıfın yapılandırmasını JSON'a dönüştürür, model kaydetme/yükleme için
    def get_config(self):
        config = {
            'patience': self.patience,
            'stop_patience': self.stop_patience,
            'threshold': self.threshold,
            'factor': self.factor,
            'batches': self.batches,
            'epochs': self.epochs,
            'ask_epoch': self.ask_epoch,
            'initial_lr': self.initial_lr,
        }
        return config

    # JSON'dan sınıfı yeniden oluşturur
    @classmethod
    def from_config(cls, config):
        return cls(**config)

    # Eğitim başladığında çağrılır, başlığı ve başlangıç zamanını ayarlar
    def on_train_begin(self, logs=None):
        # Eğitim sonuçlarını göstermek için başlık formatı
        msg = '{0:^8s}{1:^10s}{2:^9s}{3:^9s}{4:^9s}{5:^9s}{6:^9s}{7:^10s}{8:10s}{9:^8s}'.format(
            'Epoch', 'Loss', 'Accuracy', 'V_loss', 'V_acc', 'LR', 'Next LR', 'Monitor', '% Improv', 'Duration'
        )
        print(msg)
        # Eğitim başlangıç zamanını kaydeder
        self.start_time = time.time()
        # İlk ağırlıkları alır ve saklar
        self.initial_weights = self.model.get_weights()
        print(f"Training started. Initial learning rate: {self.initial_lr}")

    # Eğitim bittiğinde çağrılır, süreyi ve en iyi ağırlıkları geri yükler
    def on_train_end(self, logs=None):
        # Eğitim sonlanma zamanını alır
        stop_time = time.time()
        # Eğitim süresini hesaplar
        tr_duration = stop_time - self.start_time
        # Süreyi saat, dakika ve saniye olarak ayırır
        hours = tr_duration // 3600
        minutes = (tr_duration - (hours * 3600)) // 60
        seconds = tr_duration - ((hours * 3600) + (minutes * 60))
        # Süreyi yazdırır
        msg = f'training elapsed time was {str(hours)} hours, {minutes:4.1f} minutes, {seconds:4.2f} seconds)'
        print(msg)
        # En iyi ağırlıkları modelde geri yükler
        if self.best_weights is not None:
            self.model.set_weights(self.best_weights)
            print(f"Best weights restored from epoch {self.best_epoch}")

    # Her batch sonunda çağrılır, batch işlem durumunu yazdırır
    def on_train_batch_end(self, batch, logs=None):
        # Batch doğruluğunu ve kaybını alır
        acc = logs.get('accuracy', 0.0) * 100
        loss = logs.get('loss', float('inf'))
        # Batch işlemini yazdırır, \r ile aynı satırda günceller
        msg = '{0:20s}processing batch {1:} of {2:5s}-   accuracy=  {3:5.3f}   -   loss: {4:8.5f}'.format(
            ' ', str(batch), str(self.batches), acc, loss
        )
        print(msg, '\r', end='')

    # Her epoch başladığında çağrılır, epoch başlangıç zamanını ayarlar
    def on_epoch_begin(self, epoch, logs=None):
        self.ep_start = time.time()
        # Epoch numarasını ve toplam epoch sayısını yazdırır
        print(f"Epoch {epoch + 1}/{self.epochs} started")

    # Her epoch sonunda çağrılır, metrikleri izler ve öğrenme oranını günceller
    def on_epoch_end(self, epoch, logs=None):
        # Epoch sonlanma zamanını alır
        ep_end = time.time()
        # Epoch süresini hesaplar
        duration = ep_end - self.ep_start
        # Mevcut öğrenme oranını float olarak alır
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        current_lr = lr
        # logs None ise boş dict kullanır, metrikleri alır
        logs = logs or {}
        acc = logs.get('accuracy', 0.0)
        v_acc = logs.get('val_accuracy', 0.0)
        loss = logs.get('loss', float('inf'))
        v_loss = logs.get('val_loss', float('inf'))
    
        if acc < self.threshold:  # Eğitim doğruluğu eşikten düşükse
            monitor = 'accuracy'
            if epoch == 0:
                pimprov = 0.0
            else:
                pimprov = (acc - self.highest_tracc) * 100 / max(self.highest_tracc, 1e-10)
            
            if acc > self.highest_tracc:
                self.highest_tracc = acc
                self.best_weights = self.model.get_weights()
                self.count = 0
                self.stop_count = 0
                if v_loss < self.lowest_vloss:
                    self.lowest_vloss = v_loss
                self.best_epoch = epoch + 1
            else:
                self.count += 1
                if self.count >= self.patience:
                    new_lr = float(lr * self.factor)  # float çarpım
                    self.model.optimizer.learning_rate.assign(new_lr)  # Öğrenme oranını günceller
                    self.count = 0
                    self.stop_count += 1
                    print(f"Learning rate reduced to {new_lr}")
                    lr = new_lr  # lr'yi güncelle
                    if v_loss < self.lowest_vloss:
                        self.lowest_vloss = v_loss
    
        else:  # Eğitim doğruluğu eşikten büyükse
            monitor = 'val_loss'
            if epoch == 0:
                pimprov = 0.0
            else:
                pimprov = (self.lowest_vloss - v_loss) * 100 / max(self.lowest_vloss, 1e-10)
            
            if v_loss < self.lowest_vloss:
                self.lowest_vloss = v_loss
                self.best_weights = self.model.get_weights()
                self.count = 0
                self.stop_count = 0
                self.best_epoch = epoch + 1
                if acc > self.highest_tracc:
                    self.highest_tracc = acc
            else:
                self.count += 1
                if self.count >= self.patience:
                    new_lr = float(lr * self.factor)  # float çarpım
                    self.model.optimizer.learning_rate.assign(new_lr)  # Öğrenme oranını günceller
                    self.count = 0
                    self.stop_count += 1
                    print(f"Learning rate reduced to {new_lr}")
                    lr = new_lr  # lr'yi güncelle
                if acc > self.highest_tracc:
                    self.highest_tracc = acc
    
        if self.stop_count >= self.stop_patience:
            msg = f'training has been halted at epoch {epoch + 1} after {self.stop_patience} adjustments of learning rate with no improvement'
            print(msg)
            self.model.stop_training = True
    
        # Epoch sonuçlarını yazdırır, Epoch X/Y formatında
        msg = f'{epoch + 1}/{self.epochs:^7d}{loss:^10.4f}{acc:^9.3f}{v_loss:^9.4f}{v_acc:^9.3f}{current_lr:^9.6f}{lr:^9.6f}{monitor:^10s}{pimprov:^10.2f}{duration:^8.2f}'
        print(msg)

#### **Function to plot history of training**

In [9]:
def plot_training(hist):
    '''
    This function take training model and plot history of accuracy and losses with the best epoch in both of them.
    '''

    # Define needed variables
    tr_acc = hist.history['accuracy']
    tr_loss = hist.history['loss']
    val_acc = hist.history['val_accuracy']
    val_loss = hist.history['val_loss']
    index_loss = np.argmin(val_loss)
    val_lowest = val_loss[index_loss]
    index_acc = np.argmax(val_acc)
    acc_highest = val_acc[index_acc]
    Epochs = [i+1 for i in range(len(tr_acc))]
    loss_label = f'best epoch= {str(index_loss + 1)}'
    acc_label = f'best epoch= {str(index_acc + 1)}'

    # Plot training history
    plt.figure(figsize= (20, 8))
    plt.style.use('fivethirtyeight')

    plt.subplot(1, 2, 1)
    plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
    plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
    plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
    plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
    plt.scatter(index_acc + 1 , acc_highest, s= 150, c= 'blue', label= acc_label)
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout
    plt.show()


#### **Function to create Confusion Matrix**

In [10]:
def plot_confusion_matrix(cm, classes, normalize= False, title= 'Confusion Matrix', cmap= plt.cm.Blues):
	'''
	This function plot confusion matrix method from sklearn package.
	'''

	plt.figure(figsize= (10, 10))
	plt.imshow(cm, interpolation= 'nearest', cmap= cmap)
	plt.title(title)
	plt.colorbar()

	tick_marks = np.arange(len(classes))
	plt.xticks(tick_marks, classes, rotation= 45)
	plt.yticks(tick_marks, classes)

	if normalize:
		cm = cm.astype('float') / cm.sum(axis= 1)[:, np.newaxis]
		print('Normalized Confusion Matrix')

	else:
		print('Confusion Matrix, Without Normalization')

	print(cm)

	thresh = cm.max() / 2.
	for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
		plt.text(j, i, cm[i, j], horizontalalignment= 'center', color= 'white' if cm[i, j] > thresh else 'black')

	plt.tight_layout()
	plt.ylabel('True Label')
	plt.xlabel('Predicted Label')

# **Model Structure**

#### **Start Reading Dataset**

In [11]:
data_dir = '/kaggle/input/odir5k-classification/datasets'


try:
    # Get splitted data
    train_df, valid_df, test_df = split_data(data_dir)

    # Get Generators
    batch_size = 40
    train_gen, valid_gen, test_gen = create_gens(train_df, valid_df, test_df, batch_size)

except:
    print('Invalid Input')

Found 5113 validated image filenames belonging to 8 classes.
Found 639 validated image filenames belonging to 8 classes.
Found 640 validated image filenames belonging to 8 classes.


#### **Display Image Sample**

In [ ]:
show_images(train_gen)

#### **Generic Model Creation**

In [12]:
# Create Model Structure
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = len(list(train_gen.class_indices.keys())) # to define number of classes in dense layer

In [13]:
# set callback parameters

batch_size = 40   # set batch size for training
epochs = 30   # number of all epochs in training
patience = 5   #number of epochs to wait to adjust lr if monitored value does not improve
stop_patience = 3   # number of epochs to wait before stopping training if monitored value does not improve
threshold = 0.9   # if train accuracy is < threshold adjust monitor accuracy, else monitor validation loss
factor = 0.5   # factor to reduce lr by
ask_epoch = 5   # number of epochs to run before asking if you want to halt training
batches = int(np.ceil(len(train_gen.labels) / batch_size))    # number of training batch to run per epoch

In [ ]:
# create pre-trained model (you can built on pretrained model such as :  efficientnet, VGG , Resnet )

# we will use efficientnetb3 from EfficientNet family.
base_model = tf.keras.applications.efficientnet.EfficientNetB3(include_top= False, weights= "imagenet", input_shape= img_shape, pooling= 'max')

model = Sequential([
    base_model,
    BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    Dense(256, kernel_regularizer= regularizers.l2(0.016), activity_regularizer= regularizers.l1(0.006),
                bias_regularizer= regularizers.l1(0.006), activation= 'relu'),
    Dropout(rate= 0.45, seed= 123),
    Dense(class_count, activation= 'softmax')
])

model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model.summary()

In [ ]:
callbacks = [MyCallback(model= model, patience= patience, stop_patience= stop_patience, threshold= threshold,
            factor= factor, batches= batches, epochs= epochs, ask_epoch= ask_epoch )]

#train model
history = model.fit(x= train_gen, epochs= epochs, verbose= 0, callbacks= callbacks,
                    validation_data= valid_gen, validation_steps= None, shuffle= False)

In [ ]:
# display model performance
plot_training(history)

In [ ]:
# evaluate model
ts_length = len(test_df)
test_batch_size = test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
test_steps = ts_length // test_batch_size

train_score = model.evaluate(train_gen, steps= test_steps, verbose= 1)
valid_score = model.evaluate(valid_gen, steps= test_steps, verbose= 1)
test_score = model.evaluate(test_gen, steps= test_steps, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# ROC ve AUC çizimi için fonksiyon
def plot_roc_auc(y_test, y_pred_probs, class_count):
    if len(y_test.shape) == 1:
        y_true_onehot = tf.keras.utils.to_categorical(y_test, num_classes=class_count)
    else:
        y_true_onehot = y_test
    
    fpr = {}
    tpr = {}
    roc_auc = {}
    for i in range(class_count):
        fpr[i], tpr[i], _ = roc_curve(y_true_onehot[:, i], y_pred_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true_onehot.ravel(), y_pred_probs.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(class_count)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(class_count):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= class_count
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    
    plt.figure(figsize=(10, 8))
    for i in range(class_count):
        plt.plot(fpr[i], tpr[i], label=f'Sınıf {i} (AUC = {roc_auc[i]:.2f})')
    plt.plot(fpr["micro"], tpr["micro"], label=f'Mikro-ortalama (AUC = {roc_auc["micro"]:.2f})', linestyle='--')
    plt.plot(fpr["macro"], tpr["macro"], label=f'Makro-ortalama (AUC = {roc_auc["macro"]:.2f})', linestyle='--')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Yanlış Pozitif Oranı')
    plt.ylabel('Doğru Pozitif Oranı')
    plt.title('EfficientNetB3 ROC Eğrisi')
    plt.legend(loc="lower right")
    plt.show()

# Batch bazında tahmin yapma
preds = []
y_test = []
steps = len(test_gen)  # Test setindeki batch sayısı
for i, (batch_x, batch_y) in enumerate(test_gen):
    if i >= steps:
        break
    batch_preds = model.predict(batch_x, verbose=0)  # Her batch için tahmin
    preds.append(batch_preds)
    y_test.append(batch_y)

preds = np.concatenate(preds, axis=0)
y_test = np.concatenate(y_test, axis=0)

y_pred = np.argmax(preds, axis=1)
print("Tahmin edilen sınıf indeksleri:", y_pred)

# ROC ve AUC çizimi
plot_roc_auc(y_test, preds, class_count=len(list(train_gen.class_indices.keys())))  # Sınıf sayınızı buraya girin

In [ ]:
g_dict = test_gen.class_indices
classes = list(g_dict.keys())

# Confusion matrix
cm = confusion_matrix(test_gen.classes, y_pred)
plot_confusion_matrix(cm= cm, classes= classes, title = 'Confusion Matrix')

# Classification report
print(classification_report(test_gen.classes, y_pred, target_names= classes))

In [ ]:
# DenseNet121
densenet_model = tf.keras.applications.densenet.DenseNet121(weights='imagenet', include_top=True, input_shape= img_shape, pooling = 'max')

model = Sequential([
    densenet_model,
    BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    Dense(256, kernel_regularizer= regularizers.l2(0.016), activity_regularizer= regularizers.l1(0.006),
                bias_regularizer= regularizers.l1(0.006), activation= 'relu'),
    Dropout(rate= 0.45, seed= 123),
    Dense(class_count, activation= 'softmax')
])

model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model.summary()

In [ ]:
callbacks = [MyCallback(model= model, patience= patience, stop_patience= stop_patience, threshold= threshold,
            factor= factor, batches= batches, epochs= epochs, ask_epoch= ask_epoch )]

#train model
history = model.fit(x= train_gen, epochs= epochs, verbose= 0, callbacks= callbacks,
                    validation_data= valid_gen, validation_steps= None, shuffle= False)

In [ ]:
# display model performance
plot_training(history)

In [ ]:
# evaluate model
ts_length = len(test_df)
test_batch_size = test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
test_steps = ts_length // test_batch_size

train_score = model.evaluate(train_gen, steps= test_steps, verbose= 1)
valid_score = model.evaluate(valid_gen, steps= test_steps, verbose= 1)
test_score = model.evaluate(test_gen, steps= test_steps, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# ROC ve AUC çizimi için fonksiyon
def plot_roc_auc(y_test, y_pred_probs, class_count):
    if len(y_test.shape) == 1:
        y_true_onehot = tf.keras.utils.to_categorical(y_test, num_classes=class_count)
    else:
        y_true_onehot = y_test
    
    fpr = {}
    tpr = {}
    roc_auc = {}
    for i in range(class_count):
        fpr[i], tpr[i], _ = roc_curve(y_true_onehot[:, i], y_pred_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true_onehot.ravel(), y_pred_probs.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(class_count)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(class_count):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= class_count
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    
    plt.figure(figsize=(10, 8))
    for i in range(class_count):
        plt.plot(fpr[i], tpr[i], label=f'Sınıf {i} (AUC = {roc_auc[i]:.2f})')
    plt.plot(fpr["micro"], tpr["micro"], label=f'Mikro-ortalama (AUC = {roc_auc["micro"]:.2f})', linestyle='--')
    plt.plot(fpr["macro"], tpr["macro"], label=f'Makro-ortalama (AUC = {roc_auc["macro"]:.2f})', linestyle='--')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Yanlış Pozitif Oranı')
    plt.ylabel('Doğru Pozitif Oranı')
    plt.title('DenseNet121 ROC Eğrisi')
    plt.legend(loc="lower right")
    plt.show()

# Batch bazında tahmin yapma
preds = []
y_test = []
steps = len(test_gen)  # Test setindeki batch sayısı
for i, (batch_x, batch_y) in enumerate(test_gen):
    if i >= steps:
        break
    batch_preds = model.predict(batch_x, verbose=0)  # Her batch için tahmin
    preds.append(batch_preds)
    y_test.append(batch_y)

preds = np.concatenate(preds, axis=0)
y_test = np.concatenate(y_test, axis=0)

y_pred = np.argmax(preds, axis=1)
print("Tahmin edilen sınıf indeksleri:", y_pred)

# ROC ve AUC çizimi
plot_roc_auc(y_test, preds, class_count=len(list(train_gen.class_indices.keys())))  # Sınıf sayınızı buraya girin

In [ ]:
g_dict = test_gen.class_indices
classes = list(g_dict.keys())

# Confusion matrix
cm = confusion_matrix(test_gen.classes, y_pred)
plot_confusion_matrix(cm= cm, classes= classes, title = 'Confusion Matrix')

# Classification report
print(classification_report(test_gen.classes, y_pred, target_names= classes))

In [ ]:
# ResNet50
resnet_model = tf.keras.applications.resnet.ResNet50(weights='imagenet', include_top=True, input_shape= img_shape, pooling = 'max')

model = Sequential([
    resnet_model,
    BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    Dense(256, kernel_regularizer= regularizers.l2(0.016), activity_regularizer= regularizers.l1(0.006),
                bias_regularizer= regularizers.l1(0.006), activation= 'relu'),
    Dropout(rate= 0.45, seed= 123),
    Dense(class_count, activation= 'softmax')
])

model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model.summary()

In [ ]:
callbacks = [MyCallback(model= model, patience= patience, stop_patience= stop_patience, threshold= threshold,
            factor= factor, batches= batches, epochs= epochs, ask_epoch= ask_epoch )]

#training
history = model.fit(x= train_gen, epochs= epochs, verbose= 0, callbacks= callbacks,
                    validation_data= valid_gen, validation_steps= None, shuffle= False)

In [ ]:
# display model performance
plot_training(history)

In [ ]:
# evaluate model
ts_length = len(test_df)
test_batch_size = test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
test_steps = ts_length // test_batch_size

train_score = model.evaluate(train_gen, steps= test_steps, verbose= 1)
valid_score = model.evaluate(valid_gen, steps= test_steps, verbose= 1)
test_score = model.evaluate(test_gen, steps= test_steps, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# ROC ve AUC çizimi için fonksiyon
def plot_roc_auc(y_test, y_pred_probs, class_count):
    if len(y_test.shape) == 1:
        y_true_onehot = tf.keras.utils.to_categorical(y_test, num_classes=class_count)
    else:
        y_true_onehot = y_test
    
    fpr = {}
    tpr = {}
    roc_auc = {}
    for i in range(class_count):
        fpr[i], tpr[i], _ = roc_curve(y_true_onehot[:, i], y_pred_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true_onehot.ravel(), y_pred_probs.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(class_count)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(class_count):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= class_count
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    
    plt.figure(figsize=(10, 8))
    for i in range(class_count):
        plt.plot(fpr[i], tpr[i], label=f'Sınıf {i} (AUC = {roc_auc[i]:.2f})')
    plt.plot(fpr["micro"], tpr["micro"], label=f'Mikro-ortalama (AUC = {roc_auc["micro"]:.2f})', linestyle='--')
    plt.plot(fpr["macro"], tpr["macro"], label=f'Makro-ortalama (AUC = {roc_auc["macro"]:.2f})', linestyle='--')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Yanlış Pozitif Oranı')
    plt.ylabel('Doğru Pozitif Oranı')
    plt.title('ResNet50 ROC Eğrisi')
    plt.legend(loc="lower right")
    plt.show()

# Batch bazında tahmin yapma
preds = []
y_test = []
steps = len(test_gen)  # Test setindeki batch sayısı
for i, (batch_x, batch_y) in enumerate(test_gen):
    if i >= steps:
        break
    batch_preds = model.predict(batch_x, verbose=0)  # Her batch için tahmin
    preds.append(batch_preds)
    y_test.append(batch_y)

preds = np.concatenate(preds, axis=0)
y_test = np.concatenate(y_test, axis=0)

y_pred = np.argmax(preds, axis=1)
print("Tahmin edilen sınıf indeksleri:", y_pred)

# ROC ve AUC çizimi
plot_roc_auc(y_test, preds, class_count=len(list(train_gen.class_indices.keys())))  # Sınıf sayınızı buraya girin

In [ ]:
g_dict = test_gen.class_indices
classes = list(g_dict.keys())

# Confusion matrix
cm = confusion_matrix(test_gen.classes, y_pred)
plot_confusion_matrix(cm= cm, classes= classes, title = 'Confusion Matrix')

# Classification report
print(classification_report(test_gen.classes, y_pred, target_names= classes))

In [22]:
# MobileNetV2
mobilenet_model = tf.keras.applications.mobilenet_v2.MobileNetV2(weights='imagenet', include_top=True, input_shape= img_shape, pooling = 'max')

model = Sequential([
    mobilenet_model,
    BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    Dense(256, kernel_regularizer= regularizers.l2(0.016), activity_regularizer= regularizers.l1(0.006),
                bias_regularizer= regularizers.l1(0.006), activation= 'relu'),
    Dropout(rate= 0.45, seed= 123),
    Dense(class_count, activation= 'softmax')
])

model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 1000)                │       3,538,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 1000)                │           4,000 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │         256,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 8)                   │           2,056 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,801,296 (14.50 MB)

 Trainable params: 3,765,184 (14.36 MB)

 Non-trainable params: 36,112 (141.06 KB)

In [23]:
callbacks = [MyCallback(model= model, patience= patience, stop_patience= stop_patience, threshold= threshold,
            factor= factor, batches= batches, epochs= epochs, ask_epoch= ask_epoch )]

#training
history = model.fit(x= train_gen, epochs= epochs, verbose= 0, callbacks= callbacks,
                    validation_data= valid_gen, validation_steps= None, shuffle= False)

 Epoch     Loss   Accuracy  V_loss    V_acc     LR     Next LR  Monitor  % Improv  Duration
Training started. Initial learning rate: 0.0010000000474974513
Epoch 1/40 started
1/  40     3.8152    0.431   2.2256    0.449  0.001000 0.001000  accuracy    0.00    98.92  
Epoch 2/40 started
2/  40     2.0520    0.449   1.9651    0.449  0.001000 0.001000  accuracy    4.26    19.59  
Epoch 3/40 started
3/  40     1.9264    0.449   1.8923    0.449  0.001000 0.001000  accuracy    0.00    19.45  
Epoch 4/40 started
4/  40     1.8643    0.449   1.8368    0.449  0.001000 0.001000  accuracy    0.00    19.62  
Epoch 5/40 started
5/  40     1.8124    0.449   1.7884    0.449  0.001000 0.001000  accuracy    0.00    19.12  
Epoch 6/40 started
6/  40     1.7683    0.449   1.7487    0.449  0.001000 0.001000  accuracy    0.00    18.58  
Epoch 7/40 started
Learning rate reduced to 0.00050000002374872578  -   accuracy=  44.944   -   loss:  1.73182 
7/  40     1.7318    0.449   1.7152    0.449  0.001000 0.0005

KeyboardInterrupt: 

#### **Display model performance**

In [ ]:
plot_training(history)

# **Evaluate model**

In [ ]:
ts_length = len(test_df)
test_batch_size = test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
test_steps = ts_length // test_batch_size

train_score = model.evaluate(train_gen, steps= test_steps, verbose= 1)
valid_score = model.evaluate(valid_gen, steps= test_steps, verbose= 1)
test_score = model.evaluate(test_gen, steps= test_steps, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

# **Get Predictions**

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# ROC ve AUC çizimi için fonksiyon
def plot_roc_auc(y_test, y_pred_probs, class_count):
    if len(y_test.shape) == 1:
        y_true_onehot = tf.keras.utils.to_categorical(y_test, num_classes=class_count)
    else:
        y_true_onehot = y_test
    
    fpr = {}
    tpr = {}
    roc_auc = {}
    for i in range(class_count):
        fpr[i], tpr[i], _ = roc_curve(y_true_onehot[:, i], y_pred_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true_onehot.ravel(), y_pred_probs.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(class_count)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(class_count):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= class_count
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    
    plt.figure(figsize=(10, 8))
    for i in range(class_count):
        plt.plot(fpr[i], tpr[i], label=f'Sınıf {i} (AUC = {roc_auc[i]:.2f})')
    plt.plot(fpr["micro"], tpr["micro"], label=f'Mikro-ortalama (AUC = {roc_auc["micro"]:.2f})', linestyle='--')
    plt.plot(fpr["macro"], tpr["macro"], label=f'Makro-ortalama (AUC = {roc_auc["macro"]:.2f})', linestyle='--')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Yanlış Pozitif Oranı')
    plt.ylabel('Doğru Pozitif Oranı')
    plt.title('MobileNetV2 ROC Eğrisi')
    plt.legend(loc="lower right")
    plt.show()

# Batch bazında tahmin yapma
preds = []
y_test = []
steps = len(test_gen)  # Test setindeki batch sayısı
for i, (batch_x, batch_y) in enumerate(test_gen):
    if i >= steps:
        break
    batch_preds = model.predict(batch_x, verbose=0)  # Her batch için tahmin
    preds.append(batch_preds)
    y_test.append(batch_y)

preds = np.concatenate(preds, axis=0)
y_test = np.concatenate(y_test, axis=0)

y_pred = np.argmax(preds, axis=1)
print("Tahmin edilen sınıf indeksleri:", y_pred)

# ROC ve AUC çizimi
plot_roc_auc(y_test, preds, class_count=len(list(train_gen.class_indices.keys())))  # Sınıf sayınızı buraya girin

#### **Confusion Matrics and Classification Report**

In [ ]:
g_dict = test_gen.class_indices
classes = list(g_dict.keys())

# Confusion matrix
cm = confusion_matrix(test_gen.classes, y_pred)
plot_confusion_matrix(cm= cm, classes= classes, title = 'Confusion Matrix')

# Classification report
print(classification_report(test_gen.classes, y_pred, target_names= classes))

In [ ]:
!pip install vit-keras tensorflow-addons
from vit_keras import vit

In [ ]:
# ViT modelini yükleme
base_model = vit.vit_b16(
    image_size=224,        # Girdi boyutu
    pretrained=True,       # ImageNet ağırlıkları
    include_top=False,     # Son sınıflandırma katmanını hariç tut
    pretrained_top=False
)

# ViT modelinin çıktısı zaten [CLS] token veya havuzlanmış temsil (batch_size, 768)
# Bu nedenle, cls_token çıkarmaya gerek yok; base_model.output'u doğrudan kullanabiliriz

# Sequential model oluşturma
model = Sequential([
    base_model,
    BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
    Dense(256, kernel_regularizer=regularizers.l2(0.016), 
          activity_regularizer=regularizers.l1(0.006),
          bias_regularizer=regularizers.l1(0.006), activation='relu'),
    Dropout(rate=0.45, seed=123),
    Dense(class_count, activation='softmax')
])

# Modeli derleme
model.compile(optimizer=Adamax(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Model özetini yazdırma
model.summary()

# Create a dummy input
dummy_input = tf.random.normal((1, 224, 224, 3))

# Get the output
output = base_model(dummy_input)

print(output.shape)

In [ ]:
callbacks = [MyCallback(model= model, patience= patience, stop_patience= stop_patience, threshold= threshold,
            factor= factor, batches= batches, epochs= epochs, ask_epoch= ask_epoch )]

#training
history = model.fit(x= train_gen, epochs= epochs, verbose= 0, callbacks= callbacks,
                    validation_data= valid_gen, validation_steps= None, shuffle= False)

In [ ]:
plot_training(history)

In [ ]:
ts_length = len(test_df)
test_batch_size = test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
test_steps = ts_length // test_batch_size

train_score = model.evaluate(train_gen, steps= test_steps, verbose= 1)
valid_score = model.evaluate(valid_gen, steps= test_steps, verbose= 1)
test_score = model.evaluate(test_gen, steps= test_steps, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# ROC ve AUC çizimi için fonksiyon
def plot_roc_auc(y_test, y_pred_probs, class_count):
    if len(y_test.shape) == 1:
        y_true_onehot = tf.keras.utils.to_categorical(y_test, num_classes=class_count)
    else:
        y_true_onehot = y_test
    
    fpr = {}
    tpr = {}
    roc_auc = {}
    for i in range(class_count):
        fpr[i], tpr[i], _ = roc_curve(y_true_onehot[:, i], y_pred_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true_onehot.ravel(), y_pred_probs.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(class_count)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(class_count):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= class_count
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    
    plt.figure(figsize=(10, 8))
    for i in range(class_count):
        plt.plot(fpr[i], tpr[i], label=f'Sınıf {i} (AUC = {roc_auc[i]:.2f})')
    plt.plot(fpr["micro"], tpr["micro"], label=f'Mikro-ortalama (AUC = {roc_auc["micro"]:.2f})', linestyle='--')
    plt.plot(fpr["macro"], tpr["macro"], label=f'Makro-ortalama (AUC = {roc_auc["macro"]:.2f})', linestyle='--')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Yanlış Pozitif Oranı')
    plt.ylabel('Doğru Pozitif Oranı')
    plt.title('VitB16 ROC Eğrisi')
    plt.legend(loc="lower right")
    plt.show()

# Batch bazında tahmin yapma
preds = []
y_test = []
steps = len(test_gen)  # Test setindeki batch sayısı
for i, (batch_x, batch_y) in enumerate(test_gen):
    if i >= steps:
        break
    batch_preds = model.predict(batch_x, verbose=0)  # Her batch için tahmin
    preds.append(batch_preds)
    y_test.append(batch_y)

preds = np.concatenate(preds, axis=0)
y_test = np.concatenate(y_test, axis=0)

y_pred = np.argmax(preds, axis=1)
print("Tahmin edilen sınıf indeksleri:", y_pred)

# ROC ve AUC çizimi
plot_roc_auc(y_test, preds, class_count=len(list(train_gen.class_indices.keys())))  # Sınıf sayınızı buraya girin

In [ ]:
g_dict = test_gen.class_indices
classes = list(g_dict.keys())

# Confusion matrix
cm = confusion_matrix(test_gen.classes, y_pred)
plot_confusion_matrix(cm= cm, classes= classes, title = 'Confusion Matrix')

# Classification report
print(classification_report(test_gen.classes, y_pred, target_names= classes))

# CNN-Vision Transformer (ConvIT)

In [14]:
# Örnek parametreler
#img_shape = (224, 224, 3)  # Giriş şekli
#class_count = 10           # Sınıf sayısı (verinize göre ayarlayın)
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, LayerNormalization, MultiHeadAttention, Layer, Input, Reshape, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adamax

# CNN katmanları ile özellik çıkarımı
inputs = Input(shape=img_shape)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
cnn_output = MaxPooling2D((2, 2))(x)

In [15]:
# Özellik haritalarını patch'lere dönüştürme
sequence_length = 28 * 28  # 224 / (2^3) = 28, ama padding ile 26 olabilir, doğrulayın
embedding_dim = 128        # Her patch'in gömme boyutu (kanal sayısı)
patch_embeddings = Reshape((sequence_length, embedding_dim))(cnn_output)

In [16]:
# [CLS] token için Dense katmanını fonksiyonun dışında tanımla
cls_dense_layer = tf.keras.layers.Dense(embedding_dim, activation='relu')

def add_cls_token(inputs):
    batch_size = tf.shape(inputs)[0]  # Batch boyutunu dinamik olarak alma
    cls_token = cls_dense_layer(tf.zeros((1, 1, embedding_dim)))  # Sabit Dense katmanını kullan
    cls_token = tf.tile(cls_token, [batch_size, 1, 1])  # Batch boyutuna göre çoğaltma
    return tf.concat([cls_token, inputs], axis=1)  # [CLS] token'ini patch'lere ekleme

# Lambda katmanı ile uygulama
transformer_input = Lambda(add_cls_token)(patch_embeddings)

In [17]:
# Transformer Encoder katmanı
class TransformerEncoder(Layer):
    def __init__(self, num_heads, ff_dim, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(embedding_dim)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        attn_output = self.attention(inputs, inputs)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + ffn_output)


# Transformer encoder'ı uygulama
transformer_output = TransformerEncoder(num_heads=8, ff_dim=512)(transformer_input)

In [18]:
# [CLS] token'ini kullanarak sınıflandırma
cls_output = transformer_output[:, 0, :]  # (batch_size, embedding_dim)

# Sınıflandırma katmanları
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(cls_output)
x = Dense(256, kernel_regularizer=regularizers.l2(0.016), 
          activity_regularizer=regularizers.l1(0.006),
          bias_regularizer=regularizers.l1(0.006), activation='relu')(x)
x = Dropout(rate=0.45, seed=123)(x)
outputs = Dense(class_count, activation='softmax')(x)

In [19]:
# Tam modeli oluşturma
full_model = Model(inputs=inputs, outputs=outputs)

# Modeli derleme
full_model.compile(optimizer=Adamax(learning_rate=0.001), 
                   loss='categorical_crossentropy', 
                   metrics=['accuracy'])
# Model özetini yazdırma
full_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 224, 224, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 112, 112, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 56, 56, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 784, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda (Lambda)                      │ (None, 785, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder                  │ (None, 785, 128)            │         659,712 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ get_item (GetItem)                   │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 8)                   │           2,056 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 788,552 (3.01 MB)

 Trainable params: 788,296 (3.01 MB)

 Non-trainable params: 256 (1.00 KB)

In [20]:
# Eğitim
# Örnek callbacks 
callbacks = [MyCallback(model= full_model, patience= patience, stop_patience= stop_patience, threshold= threshold,
            factor= factor, batches= batches, epochs= epochs, ask_epoch= ask_epoch )]

history = full_model.fit(x=train_gen, epochs=epochs, verbose=0, callbacks=callbacks,
                         validation_data=valid_gen, validation_steps=None, shuffle=False)

 Epoch     Loss   Accuracy  V_loss    V_acc     LR     Next LR  Monitor  % Improv  Duration
Training started. Initial learning rate: 0.0010000000474974513
Epoch 1/40 started
1/  40     5.1318    0.428   13.4016   0.449  0.001000 0.001000  accuracy    0.00    96.86  
Epoch 2/40 started


KeyboardInterrupt: 

In [ ]:
plot_training(history)

In [ ]:
ts_length = len(test_df)
test_batch_size = test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
test_steps = ts_length // test_batch_size

train_score = full_model.evaluate(train_gen, steps= test_steps, verbose= 1)
valid_score = full_model.evaluate(valid_gen, steps= test_steps, verbose= 1)
test_score = full_model.evaluate(test_gen, steps= test_steps, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# ROC ve AUC çizimi için fonksiyon
def plot_roc_auc(y_test, y_pred_probs, class_count):
    if len(y_test.shape) == 1:
        y_true_onehot = tf.keras.utils.to_categorical(y_test, num_classes=class_count)
    else:
        y_true_onehot = y_test
    
    fpr = {}
    tpr = {}
    roc_auc = {}
    for i in range(class_count):
        fpr[i], tpr[i], _ = roc_curve(y_true_onehot[:, i], y_pred_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true_onehot.ravel(), y_pred_probs.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(class_count)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(class_count):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= class_count
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    
    plt.figure(figsize=(10, 8))
    for i in range(class_count):
        plt.plot(fpr[i], tpr[i], label=f'Sınıf {i} (AUC = {roc_auc[i]:.2f})')
    plt.plot(fpr["micro"], tpr["micro"], label=f'Mikro-ortalama (AUC = {roc_auc["micro"]:.2f})', linestyle='--')
    plt.plot(fpr["macro"], tpr["macro"], label=f'Makro-ortalama (AUC = {roc_auc["macro"]:.2f})', linestyle='--')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Yanlış Pozitif Oranı')
    plt.ylabel('Doğru Pozitif Oranı')
    plt.title('ConvIT ROC Eğrisi')
    plt.legend(loc="lower right")
    plt.show()

# Batch bazında tahmin yapma
preds = []
y_test = []
steps = len(test_gen)  # Test setindeki batch sayısı
for i, (batch_x, batch_y) in enumerate(test_gen):
    if i >= steps:
        break
    batch_preds = full_model.predict(batch_x, verbose=0)  # Her batch için tahmin
    preds.append(batch_preds)
    y_test.append(batch_y)

preds = np.concatenate(preds, axis=0)
y_test = np.concatenate(y_test, axis=0)

y_pred = np.argmax(preds, axis=1)
print("Tahmin edilen sınıf indeksleri:", y_pred)

# ROC ve AUC çizimi
plot_roc_auc(y_test, preds, class_count=len(list(train_gen.class_indices.keys())))  # Sınıf sayınızı buraya girin

In [ ]:
g_dict = test_gen.class_indices
classes = list(g_dict.keys())

# Confusion matrix
cm = confusion_matrix(test_gen.classes, y_pred)
plot_confusion_matrix(cm= cm, classes= classes, title = 'Confusion Matrix')

# Classification report
print(classification_report(test_gen.classes, y_pred, target_names= classes))